#受講証明用課題①回答欄

In [ ]:
import random
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
from qulacs import QuantumCircuit, QuantumState
from qulacs.gate import CNOT, H, RZ, RX, RY, S, Sdag, RandomUnitary
from qulacs.circuit import ParametricQuantumCircuit
from qulacs.state import inner_product

# ランダムユニタリーゲートの作成
target_list = [0, 1]
gate = RandomUnitary(target_list)
print(gate)

# パラメトリック量子回路の作成
nqubits = 2
anzatz = ParametricQuantumCircuit(nqubits)
anzatz.add_parametric_RZ_gate(0, 0.0)
anzatz.add_parametric_RZ_gate(1, 0.0)
anzatz.add_parametric_RX_gate(0, 0.0)
anzatz.add_parametric_RX_gate(1, 0.0)
anzatz.add_parametric_RZ_gate(0, 0.0)
anzatz.add_parametric_RZ_gate(1, 0.0)
anzatz.add_CNOT_gate(0, 1)
anzatz.add_parametric_RX_gate(0, 0.0)
anzatz.add_parametric_RZ_gate(1, 0.0)
anzatz.add_CNOT_gate(0, 1)
anzatz.add_S_gate(1)
anzatz.add_CNOT_gate(0, 1)
anzatz.add_parametric_RY_gate(0, 0.0)
anzatz.add_CNOT_gate(0, 1)
anzatz.add_Sdag_gate(1)
anzatz.add_parametric_RZ_gate(0, 0.0)
anzatz.add_parametric_RZ_gate(1, 0.0)
anzatz.add_parametric_RX_gate(0, 0.0)
anzatz.add_parametric_RX_gate(1, 0.0)
anzatz.add_parametric_RZ_gate(0, 0.0)
anzatz.add_parametric_RZ_gate(1, 0.0)
anzatz.add_gate(gate)

# 量子状態の準備
mes = QuantumState(nqubits)
H(0).update_quantum_state(mes)
CNOT(0, 1).update_quantum_state(mes)
mes_copy = mes.copy()

# パラメトリック回路の適用
anzatz.update_quantum_state(mes)

# エンタングルメント忠実度の計算
ent_fidelity = abs(inner_product(mes_copy, mes))**2

# コスト関数の定義
def cost(theta_list):
    # パラメータを設定
    anzatz.set_parameter_vector(theta_list)
    # 量子状態をリセットして再計算
    mes = QuantumState(nqubits)
    H(0).update_quantum_state(mes)
    CNOT(0, 1).update_quantum_state(mes)
    anzatz.update_quantum_state(mes)
    # エンタングルメント忠実度の計算
    ent_fidelity = abs(inner_product(mes_copy, mes))**2
    # コストとして1 - 忠実度を返す
    return 1 - ent_fidelity

# コスト履歴の保存
cost_history = []

# パラメータの初期値
init_theta_list = [random.random() for i in range(anzatz.get_parameter_count())]
cost_history.append(cost(init_theta_list))

# 最適化
method = "BFGS"
options = {"disp": True, "maxiter": 50, "gtol": 1e-6}
opt = scipy.optimize.minimize(cost, init_theta_list,
               method=method,
               callback=lambda x: cost_history.append(cost(x)),
               options=options)

# コスト履歴のプロット
plt.rcParams["font.size"] = 18
plt.plot(cost_history)
plt.xlabel("Iteration")
plt.ylabel("1 - (Entanglement Fidelity)")
plt.show()

# 最適化されたパラメータ
opt_theta = opt.x
print("Optimized Parameters:", opt_theta)


#グループワーク参加資格用課題①回答欄